# Activity recognition on the Capture24 dataset -- Long short-term memory

*This section assumes familiarity with [PyTorch](https://pytorch.org/)*

We have seen the importance of accounting for the temporal dependencies in the data to improve the classification accuracy. Here we look at using a more flexible model -- the Long short-term memory -- to improve performance.

Setup:

In [ ]:
import numpy as np
from scipy.ndimage import median_filter
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from tqdm import tqdm
import utils

# For reproducibility
np.random.seed(42)
torch.manual_seed(42)
cudnn.benchmark = True

 Load dataset and hold out some instances for testing: 

In [ ]:
data = np.load('capture24_small.npz', allow_pickle=True)
print("Data contents:", data.files)
X, y, pid, time = data['X_feats'], data['y'], data['pid'], data['time']

# Hold out some participants for testing the model
test_pids = [2, 3]
test_mask = np.isin(pid, test_pids)
train_mask = ~np.isin(pid, test_pids)
X_train, y_train, pid_train = X[train_mask], y[train_mask], pid[train_mask]
X_test, y_test, pid_test = X[test_mask], y[test_mask], pid[test_mask]

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

In [ ]:
classifier = RandomForestClassifier(n_estimators=100, n_jobs=2)
classifier.fit(X_train, y_train)
Y_train_pred = classifier.predict_proba(X_train)
# Y_train = utils.encode_one_hot(y_train)

Y_train_pred_torch = torch.from_numpy(Y_train_pred.astype('float32'))
# Y_train_torch = torch.from_numpy(Y_train.astype('float32'))
y_train_torch = torch.from_numpy(y_train)

# testing
#TODO: function that windows through X_train, X_test
#TODO: batch sizing

 Define a dataset iterator that will provide the mini-batches during training 

In [ ]:
def create_dataloader(batch_size=1):
    # for i in range(2,len(Y_train_pred_torch)):
    #     sequence = torch.stack((
    #         Y_train_pred_torch[i-2],
    #         Y_train_pred_torch[i-1],
    #         Y_train_pred_torch[i]
    #     )).view(3, 1, -1)
    #     target = y_train_torch[i].view(1)
    #     yield sequence, target
    Y = torch.stack((
        Y_train_pred_torch[:-2],
        Y_train_pred_torch[1:-1],
        Y_train_pred_torch[2:]
    ))
    nbatch = Y.shape[1]//batch_size
    idxs = np.random.permutation(np.arange(Y.shape[1]))
    for i in range(nbatch):
        # batch = []
        # for j in range(batch_size):
        #     batch.append(Y[:,i].view(3, 1, -1))
        # batch = torch.cat(batch, dim=1)
        # sequence_batch = Y[:,i:i+batch_size]
        # target_batch = y_train_torch[i:i+batch_size]
        _idxs = idxs[i:i+batch_size]
        sequence_batch = Y[:,_idxs]
        target_batch = y_train_torch[_idxs]
        yield sequence_batch, target_batch

 Define a simple LSTM model

In [ ]:
class LSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, output_dim):
        super(LSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2output = nn.Linear(hidden_dim, output_dim)

    def forward(self, sequence):
        _, (hidden_last, cell_last) = self.lstm(
            sequence.view(len(sequence), -1, self.embedding_dim))
        output = self.hidden2output(hidden_last.view(-1, self.hidden_dim))
        # output = F.log_softmax(output, dim=1)
        return output

 Instantiate LSTM model, define loss function and optimizer 

In [ ]:
HIDDEN_DIM = 4
lstm = LSTM(utils.NUM_CLASSES, HIDDEN_DIM, utils.NUM_CLASSES)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm.parameters(), lr=1e-2, amsgrad=True)

 Training via mini-batch gradient descent begins here. We loop through the training set `max_iter` times with the `dataloader` iterator.

In [ ]:
max_iter = 100
batch_size = 32
losses = []
for i in tqdm(range(max_iter)):
    dataloader = create_dataloader(batch_size)
    for sequence, target in dataloader:
        lstm.zero_grad()
        output = lstm(sequence)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

utils.plt.semilogy(median_filter(losses, size=100))
utils.plt.savefig('lstm_loss.png')

 Evaluate the model on the hold out set 

In [ ]:
print("Testing...")
lstm.eval()
Y_test_pred = classifier.predict_proba(X_test)
Y_test_pred_torch = torch.from_numpy(Y_test_pred.astype('float32'))
y_test_pred_lstm = [np.argmax(Y_test_pred[0]), np.argmax(Y_test_pred[1])]
with torch.no_grad():
    for i in range(2,len(Y_test_pred_torch)):
        sequence = torch.stack((
            Y_test_pred_torch[i-2],
            Y_test_pred_torch[i-1],
            Y_test_pred_torch[i]
        )).view(3, 1, -1)
        y_test_pred_lstm.append(torch.argmax(F.softmax(lstm(sequence), dim=-1)).item())
y_test_pred_lstm = np.asarray(y_test_pred_lstm)
print("Cohen kappa score:", utils.cohen_kappa_score(y_test, y_test_pred_lstm, pid_test))
print("Accuracy score:", utils.accuracy_score(y_test, y_test_pred_lstm, pid_test))
